## Folder Path

In [1]:
input_folder_path = "../../Input/csv_files/"

## Libraries Import

In [2]:
from neo4j import GraphDatabase
import pandas as pd
import os

## Data Loading

In [3]:
df = pd.read_csv(os.path.abspath(os.path.join(input_folder_path, "sample_cast_1.csv")))

## Neo4j Config

In [4]:
# Load config from file
def load_neo4j_config(filepath):
    config = {}
    with open(filepath, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

In [5]:
config = load_neo4j_config("../Encryption/Neo4j-1cf46bb6-Created-2025-04-10.txt")

In [6]:
uri = config.get("NEO4J_URI")
username = config.get("NEO4J_USERNAME")
password = config.get("NEO4J_PASSWORD")

## Driver Initialisation

In [7]:
# Create driver
driver = GraphDatabase.driver(uri, auth=(username, password))

## Test Connection

In [8]:
# Example query to test
def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Connection successful!' AS message")
        print(result.single()["message"])

In [9]:
test_connection()

Connection successful!


## Data Loading

In [10]:
def load_data(tx, personId, name, birthYear):
    query = """
        MERGE (p: Person {personId: $personId})
        SET p.name = $name, p.birthYear = $birthYear
    """
    
    tx.run(query, personId=personId, name=name, birthYear=birthYear)

In [11]:
with driver.session() as session:
    for index, row in df.iterrows():
        session.execute_write(load_data, row['personId'], row['name'], row['birthYear'])
        
print("CSV file successfully loaded into Neo4j.")

CSV file successfully loaded into Neo4j.


## View Data

In [12]:
def view_data(tx):
    query = """
        MATCH (p:Person)
        RETURN p.personId AS personId, p.name AS name, p.birthYear AS birthYear
    """
    result = tx.run(query)
    
    return [record.data() for record in result]

In [13]:
with driver.session() as session:
    people = session.execute_read(view_data)
    
    for person in people:
        print(person)

{'personId': 23945, 'name': 'Gerard Pires', 'birthYear': 1942}
{'personId': 553509, 'name': 'Helen Reddy', 'birthYear': 1941}
{'personId': 113934, 'name': 'Susan Flannery', 'birthYear': 1939}


## Load Online CSV Files

In [14]:
## Load Online CSV Files
def load_online_data(tx):
    query = """
        LOAD CSV WITH HEADERS
        FROM "https://data.neo4j.com/importing-cypher/persons.csv" AS row
        RETURN row
    """
    result = tx.run(query)
    
    # Consume the result inside the transaction
    return [record["row"] for record in result]

In [15]:
with driver.session() as session:
    people = session.execute_read(load_online_data)

    # Now this is safe, because the data was returned from within the open transaction
    for person in people:
        print(person)

{'born': '1942-07-13', 'bornIn': 'Chicago, Illinois, USA', 'bio': 'Legendary Hollywood Icon Harrison Ford was born on July 13,\xa01942 in Chicago, Illinois. \xa0 His family\xa0history includes a strong lineage of actors, radio personalities, and models. \xa0 Harrison attended public high school in Park\xa0Ridge, Illinois where he was a member of the school Radio Station WMTH. \xa0Harrison worked as the lead voice for sports reporting at WMTH for several\xa0years. \xa0 Acting wasn’t a major interest to\xa0Ford until his junior year at Ripon College when he first took an acting class...', 'died': None, 'person_url': 'https://themoviedb.org/person/3', 'name': 'Harrison Ford', 'person_poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/5M7oN3sznp99hWYQ9sX0xheswWX.jpg', 'person_imdbId': '148', 'person_tmdbId': '3'}
{'born': '1956-07-09', 'bornIn': 'Concord, California, USA', 'bio': 'Thomas Jeffrey Hanks (born July 9, 1956) is an American actor and filmmaker.  Known for both his comedic 

In [16]:
def load_actors_1940s(tx):
    query = """
        LOAD CSV WITH HEADERS
        FROM "https://data.neo4j.com/importing-cypher/persons.csv" AS row
        WITH row
        WHERE toInteger(substring(row.born, 0, 4)) >= 1940 AND toInteger(substring(row.born, 0, 4)) < 1950
        RETURN row
    """
    result = tx.run(query)
    return [record["row"] for record in result]

In [17]:
with driver.session() as session:
    actors_1940s = session.execute_read(load_actors_1940s)

    for actor in actors_1940s:
        print(actor)

{'born': '1942-07-13', 'bornIn': 'Chicago, Illinois, USA', 'bio': 'Legendary Hollywood Icon Harrison Ford was born on July 13,\xa01942 in Chicago, Illinois. \xa0 His family\xa0history includes a strong lineage of actors, radio personalities, and models. \xa0 Harrison attended public high school in Park\xa0Ridge, Illinois where he was a member of the school Radio Station WMTH. \xa0Harrison worked as the lead voice for sports reporting at WMTH for several\xa0years. \xa0 Acting wasn’t a major interest to\xa0Ford until his junior year at Ripon College when he first took an acting class...', 'died': None, 'person_url': 'https://themoviedb.org/person/3', 'name': 'Harrison Ford', 'person_poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/5M7oN3sznp99hWYQ9sX0xheswWX.jpg', 'person_imdbId': '148', 'person_tmdbId': '3'}
{'born': '1946-11-06', 'bornIn': 'Pasadena, California, USA', 'bio': 'Sally Margaret Field (born November 6, 1946) is an American actress and director.  She is the recipient 

## Close Driver

In [ ]:
driver.close()